In [1]:
#export
"""This module for tools that will likely start the processing stream."""
from typing import Iterator, Union
import urllib, subprocess, warnings, os
from k1lib.cli import BaseCli
import k1lib.cli as cli
__all__ = ["cat", "cats", "curl", "wget", "ls", "cmd", "requireCli", "toPIL"]

In [2]:
#export
def _catSimple(fileName:str=None, text:bool=True) -> Iterator[Union[str, bytes]]:
    if text:
        with open(fileName) as f:
            for line in f.readlines():
                if line[-1] == "\n": yield line[:-1]
                else: yield line
    else:
        with open(fileName, "rb") as f: yield f.read()
def _catWrapper(fileName:str, text:bool):
    res = _catSimple(fileName, text)
    return res if text else next(res)
class _cat(BaseCli):
    def __init__(self, text): self.text = text
    def __ror__(self, fileName:str) -> Union[Iterator[str], bytes]:
        return _catWrapper(fileName, self.text)
def cat(fileName:str=None, text:bool=True):
    """Reads a file line by line.
Example::

    # display first 10 lines of file
    cat("file.txt") | headOut()
    # piping in also works
    "file.txt" | cat() | headOut()
    
    # rename file
    cat("img.png", False) | file("img2.png", False)

:param fileName: if None, then return a :class:`~k1lib.cli.init.BaseCli`
    that accepts a file name and outputs Iterator[str]
:param text: if True, read text file, else read binary file"""
    if fileName is None: return _cat(text)
    else: return _catWrapper(fileName, text)

In [3]:
assert len("inp.py" | cat() | cli.head() | cli.toList()) == 10
assert len(cat("inp.py") | cli.head() | cli.toList()) == 10

In [4]:
#export
class cats(BaseCli):
    """Like :meth:`cat`, but opens multiple files at once, returning
streams. Looks something like this::

    apply(lambda s: cat(s))

Example::

    # prints out first 10 lines of 2 files
    ["file1.txt", "file2.txt"] | cats() | headOut().all() | ignore()"""
    def __ror__(self, fileNames:Iterator[str]) -> Iterator[Iterator[str]]:
        for fileName in fileNames:
            yield _catSimple(fileName)

In [5]:
assert ["inp.py", "init.py"] | cats() | (cli.head() | cli.toList()).all() | cli.deref() | cli.shape() == (2, 10)

In [6]:
#export
def curl(url:str) -> Iterator[str]:
    """Gets file from url. File can't be a binary blob.
Example::

    # prints out first 10 lines of the website
    curl("https://k1lib.github.io/") | headOut()"""
    for line in urllib.request.urlopen(url):
        line = line.decode()
        if line[-1] == "\n": yield line[:-1]
        else: yield line
def wget(url:str, fileName:str=None):
    """Downloads a file

:param url: The url of the file
:param fileName: if None, then tries to infer it from the url"""
    if fileName is None: fileName = url.split("/")[-1]
    urllib.request.urlretrieve(url, fileName)

In [7]:
assert curl("https://k1lib.github.io/") | cli.shape(0) > 0

In [8]:
#export
def ls(folder:str=None):
    """List every file and folder inside the specified folder.
Example::

    # returns List[str]
    ls("/home")
    # same as above
    "/home" | ls()
    # only outputs files, not folders
    ls("/home") | isFile()

See also: :meth:`~k1lib.cli.filt.isFile`"""
    if folder is None: return _ls()
    else: return folder | _ls()
class _ls(BaseCli):
    def __ror__(self, folder:str):
        return [f"{folder}/{e}" for e in os.listdir(folder)]

In [9]:
assert len(ls("/home")) == len("/home" | ls())
assert len(ls("/home")) > 0
assert len("/home/kelvin" | ls() | cli.isFile() | cli.deref()) > 0

In [10]:
#export
def executeCmd(cmd:str, inp:str=None):
    """Runs a command, and returns the output line by line"""
    if inp is None:
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out, err = process.communicate()
    else:
        process = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out, err = process.communicate(input=inp.encode("utf-8"))
    return out.decode().split("\n"), err
class cmd(BaseCli):
    def __init__(self, cmd:str):
        """Runs a command, and returns the output line by line.
Example::

    # return detailed list of files
    None | cmd("ls -la")
    # return list of files that ends with "ipynb"
    None | cmd("ls -la") | cmd('grep ipynb$')"""
        super().__init__(); self.cmd = cmd; self.err = b''
    @property
    def err(self) -> bytes:
        """Error from the last command"""
        return self._err
    @err.setter
    def err(self, value): self._err = value
    def __ror__(self, it:Union[Iterator[str], None]) -> Iterator[str]:
        """Pipes in lines of input, or if there's nothing to
pass, then pass None"""
        super().__ror__(it)
        out, err = executeCmd(self.cmd) if it is None else executeCmd(self.cmd, it | cli.to1Str("\n") | cli.item())
        if err: warnings.warn(f"Error encountered:\n\n{err.decode()}")
        self._err = err; return out
    def __repr__(self):
        return """`k1lib.cli.cmd` instance. Pipes
input generator or None in to execute."""
def requireCli(cliTool:str):
    """Searches for a particular cli tool (eg. "ls"), throws ImportError
if not found, else do nothing"""
    a = cmd(cliTool); None | a;
    if len(a.err) > 0:
        raise ImportError(f"""Can't find cli tool {cliTool}. Please install
it first.""")

In [11]:
assert None | cmd("ls -la") | cli.shape(0) > 0
assert None | cmd("ls -la") | cmd('grep ipynb$') | cli.shape(0) > 0

In [12]:
#export
class toPIL(BaseCli):
    def __init__(self):
        """Converts a path to a PIL image.
Example::

    ls(".") | toPIL().all() | item() # get first image"""
        import PIL; self.PIL = PIL
    def __ror__(self, path) -> "PIL.Image.Image":
        return self.PIL.Image.open(path)

In [13]:
!../../export.py cli/inp

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.5
Uninstalling k1lib-0.5:
  Successfully uninstalled k1lib-0.5
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/fmt.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib